# Load database

In [1]:
from datetime import datetime
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import os
from dotenv import load_dotenv

In [2]:
# Carregar variáveis de ambiente do arquivo .env
load_dotenv()

# Obter caminhos do arquivo csv e do diretório de extração das variáveis de ambiente
transacoes_path = os.getenv('TRANSACOES_CSV')

In [3]:
# Carregar o CSV em um DataFrame Pandas
df_transacoes = pd.read_csv(transacoes_path)

**Wrangling**

Vamos realizar a manipulação dos dados para o dashboard, priorizando a obtenção de uma granularidade consistente e aprimoramento da qualidade da base de dados.

In [4]:
df_transacoes.head()

,cod_transacao,num_conta,data_transacao,nome_transacao,valor_transacao
0,3100,256,2020-05-22 03:15:21 UTC,Saque,-130.0
1,28405,256,2021-06-29 11:52:38 UTC,Saque,-255.0
2,1776,512,2020-05-17 13:04:50 UTC,Saque,-250.0
3,14311,512,2020-09-06 02:55:34 UTC,Saque,-80.0
4,25012,512,2020-10-16 09:26:52 UTC,Saque,-475.0


In [5]:
df_transacoes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71999 entries, 0 to 71998
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   cod_transacao    71999 non-null  int64  
 1   num_conta        71999 non-null  int64  
 2   data_transacao   71999 non-null  object 
 3   nome_transacao   71999 non-null  object 
 4   valor_transacao  71999 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 2.7+ MB


Vamos fazer um type casting da coluna data_transacao para o formato de 'datetime64[ns]'

A coluna data_transacao está formatada com o sufixo "UTC". Para lidar com isso, usaremos a função pd.to_datetime() do pandas, que oferece mais flexibilidade para o parsing de strings de data.

In [6]:
# Função para corrigir e converter datas
def corrigir_e_converter(data):
    try:
        return pd.to_datetime(data, errors='coerce')
    except ValueError:
        return pd.NaT

# Aplicar a função para corrigir e converter a coluna 'data_transacao'
df_transacoes['data_transacao'] = df_transacoes['data_transacao'].apply(corrigir_e_converter)

# Verificar a conversão
print(df_transacoes['data_transacao'])



0              2020-05-22 03:15:21+00:00
1              2021-06-29 11:52:38+00:00
2              2020-05-17 13:04:50+00:00
3              2020-09-06 02:55:34+00:00
4              2020-10-16 09:26:52+00:00
                      ...               
71994   2022-12-29 23:59:59.746608+00:00
71995   2022-12-29 23:59:59.055028+00:00
71996          2012-05-07 13:45:41+00:00
71997          2015-09-19 05:13:07+00:00
71998   2022-12-29 23:59:59.607373+00:00
Name: data_transacao, Length: 71999, dtype: datetime64[ns, UTC]


In [7]:
print(df_transacoes['data_transacao'].isna().any())

False


In [8]:
# Criar a coluna para a data
df_transacoes['transacao_date'] = df_transacoes['data_transacao'].dt.date

# Criar a coluna para a hora
df_transacoes['transacao_time'] = df_transacoes['data_transacao'].dt.time

# Verificar os resultados
print(df_transacoes[['transacao_date', 'transacao_time']])


      transacao_date   transacao_time
0         2020-05-22         03:15:21
1         2021-06-29         11:52:38
2         2020-05-17         13:04:50
3         2020-09-06         02:55:34
4         2020-10-16         09:26:52
...              ...              ...
71994     2022-12-29  23:59:59.746608
71995     2022-12-29  23:59:59.055028
71996     2012-05-07         13:45:41
71997     2015-09-19         05:13:07
71998     2022-12-29  23:59:59.607373

[71999 rows x 2 columns]


Enriquecimento dos dados com novas colunas.

Chaves temporais:

In [9]:
df_transacoes['transacao_date_month'] = df_transacoes['transacao_date'].apply(lambda date: date.strftime('%Y-%m'))
df_transacoes['transacao_date_year']  = df_transacoes['transacao_date'].apply(lambda date: date.strftime('%Y'))

Com os dados manipulados, procederemos à sua persistência em disco, realizaremos o download e, posteriormente, o carregaremos no Google Data Studio.

In [10]:
# Obter caminhos do diretório gold das variáveis de ambiente
gold_path = os.getenv('GOLD_PATH')

In [11]:
df_transacoes.to_csv(f'{gold_path}/transacoes.csv', sep=',', index=False)